# Prototype of a flexible scheduler

We define an event which calls a function at a given timestamp. An event can schedule the next event. The events can be repetitive or nested, scheduling themselves $N$ times.

In [1]:
from time import time, sleep
from random import random

In [2]:
t0 = time()

def t():
    global t0
    return time() - t0

In [3]:
# Global definitions

open_shutters  = lambda : "opening shutters"
close_shutters = lambda : "closing shutters"
camera_up      = lambda : "camera ↑"
camera_dn      = lambda : "camera  ↓"
cy2_laser      = lambda : " Cy2"
cy3_laser      = lambda : "   Cy3"
cy5_laser      = lambda : "     Cy5"
cy7_laser      = lambda : "       Cy7"
fluidics_up    = lambda : " ===== fluidics ↑"
fluidics_dn    = lambda : " ===== fluidics  ↓"
bullsh         = lambda : " This butt (_!_) shouldn't be here if your script works correctly"
easter_egg     = lambda : "SURPRISE event [="

# Global vars for one-time events (ote_)
class OTE:
    N_events = 0
    idx = 0
ote = OTE()

# Global vars for repetitive events (re_)
re_started = 0
class RE:
    N_cycles = 0
    cycle_interval = 0
    cycle_ctr = 0
    N_events = 0
    idx = 0
re = RE()

class Event():
    def __init__(self, timestamp, func):
        self.timestamp = timestamp
        self.func = func
    def call(self):
        print(f"{self.timestamp:7.3f}:  ", self.func())
    def __repr__(self):
        return f"Event {self.timestamp:7.3f}: {self.func()}"

rep_events = [Event(20*random(), bullsh) for i in range(100)]
one_time_events = [Event(20*random(), bullsh) for i in range(100)]
final_event = None

In [4]:
# Prepare data acquisition
def init_ote():
    ote.N_events = 5
    ote.idx = 0

def init_re():
    re.N_cycles = 5
    re.cycle_interval = 2
    re.cycle_ctr = 0
    re.N_events = 12
    re.idx = 0

init_ote()
init_re()
re_started = 0

def start_rep_events():
    global re_started
    re_started = t()
    
    # Update timestamps for current time. In practice, we will just start another timer/counter
    for event in rep_events:
        event.timestamp += re_started
    
    return f"Starting repeating events, N={re.N_cycles}"

# Define events
for i, l in enumerate([
    Event(0, cy2_laser),
    Event(0.05, camera_up),
    Event(0.25, camera_dn),

    Event(0.5, cy3_laser),
    Event(0.55, camera_up),
    Event(0.75, camera_dn),

    Event(1, cy5_laser),
    Event(1.05, camera_up),
    Event(1.25, camera_dn),

    Event(1.5, cy7_laser),
    Event(1.55, camera_up),
    Event(1.75, camera_dn),
]):
    rep_events[i] = l


for i, l in enumerate([
    Event(0.1, open_shutters),
    Event(0.4, start_rep_events),
    Event(1.3, fluidics_up),
    Event(3.35, fluidics_dn),
    Event(2+7*random(), easter_egg),
]):
    one_time_events[i] = l
    
final_event = lambda : print(close_shutters() + " -> end of data acquisition")

In [5]:
# Cyclic event interrupt - should process one event only, and reschedule it
def re_IRQ():
    if re.cycle_ctr < re.N_cycles:
        if re.idx < re.N_events:
            rep_events[re.idx].call()
            rep_events[re.idx].timestamp += re.cycle_interval  # reschedule it
            re.idx += 1
        if  re.idx == re.N_events:
            re.idx = 0
            re.cycle_ctr += 1
        return
    global re_started
    re_started = 0
    final_event()
    
def ote_IRQ():
    one_time_events[ote.idx].call()
    ote.idx += 1

In [6]:
t0 = time()
init_ote()

while t() < 20: # ote.idx < ote.N_events:
    if t() >= one_time_events[ote.idx].timestamp and ote.idx < ote.N_events:
        ote_IRQ()
    if re_started and (t() >= rep_events[re.idx].timestamp):
        re_IRQ()
    sleep(0.01)

  0.100:   opening shutters
  0.400:   Starting repeating events, N=5
  0.406:    Cy2
  0.456:   camera ↑
  0.656:   camera  ↓
  0.906:      Cy3
  0.956:   camera ↑
  1.156:   camera  ↓
  1.300:    ===== fluidics ↑
  1.406:        Cy5
  1.456:   camera ↑
  1.656:   camera  ↓
  1.906:          Cy7
  1.956:   camera ↑
  2.156:   camera  ↓
  2.406:    Cy2
  2.456:   camera ↑
  2.656:   camera  ↓
  2.906:      Cy3
  2.956:   camera ↑
  3.156:   camera  ↓
  3.350:    ===== fluidics  ↓
  3.406:        Cy5
  3.456:   camera ↑
  3.656:   camera  ↓
  3.906:          Cy7
  3.956:   camera ↑
  4.156:   camera  ↓
  4.406:    Cy2
  4.456:   camera ↑
  4.656:   camera  ↓
  4.906:      Cy3
  4.956:   camera ↑
  5.156:   camera  ↓
  5.406:        Cy5
  5.456:   camera ↑
  5.656:   camera  ↓
  5.906:          Cy7
  5.956:   camera ↑
  6.156:   camera  ↓
  6.406:    Cy2
  6.456:   camera ↑
  6.656:   camera  ↓
  6.906:      Cy3
  6.956:   camera ↑
  7.156:   camera  ↓
  7.406:        Cy5
  7.456:   came